In [22]:
import numpy as np
import torch
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision.datasets import ImageFolder
import glob
import pickle

#if torch.cuda.is_available():
#  print("Cuda (GPU support) is available and enabled!")
#  device = torch.device("cuda")
#else:
#  print("Cuda (GPU support) is not available :(")
device = torch.device("cpu")

In [3]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

resnet50.eval().to(device)
resnet50 = resnet50.to(device)

Using cache found in /home/umut/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/umut/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/umut/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
Using cache found in /home/umut/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [4]:
feature_path = "/home/umut/Desktop/501/others/Export/"
image_filenames = os.listdir(feature_path)[:2880]
features = []

In [5]:
nodes, _ = get_graph_node_names(resnet50)

return_nodes = {
    'layers.0.0.downsample.1': 'layer1',
    'layers.1.0.downsample.1': 'layer2',
    'layers.2.0.downsample.1': 'layer3'
}


extractor = create_feature_extractor(resnet50, return_nodes=return_nodes)

In [24]:
all_tensors = torch.Tensor(np.zeros(shape=(len(image_filenames), 3, 480, 640)))
idx = 0
for idx, filename in enumerate(image_filenames):
    image = Image.open(feature_path + filename)
    
    preprocess = transforms.Compose([
    transforms.Resize((480, 640)),
    transforms.ToTensor()
    ])
    input_tensor = preprocess(image)
    all_tensors[idx] = input_tensor
    idx += 1

In [16]:
features = extractor(all_tensors.to(device)) # Unable to run this with all the images!

In [17]:
print(features.keys())

dict_keys(['layer1', 'layer2', 'layer3'])


In [23]:
with open('data/features.pkl', 'wb') as f:
    pickle.dump(features, f)